# 🚀 Welcome to Your ADK Adventure - Tools & Memory! 🚀

Part 0: Setup & Authentication 🔑

In [1]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


/home/oem/Repos/CursoLLM/venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [2]:
api_key = getpass('Enter your Google API Key: ')

genai.configure(api_key=api_key)

os.environ['GOOGLE_API_KEY'] = api_key

genai.configure(api_key=api_key)
print("✅ API Key configured successfully! Let the fun begin.")

✅ API Key configured successfully! Let the fun begin.


In [7]:
# -- Agent Definition

# This function creates a new AI agent called "weekend_trip_agent"
def create_weekend_trip_agent():
  """Create the Spontaneous Day Trip Generator agent"""
  return Agent(
      name="weekend_trip_agent", # Internal name of the agent
      model="gemini-2.5-flash",  #The LLM powering the agent (fast + cheap)
      description="Agent specialized in generating spontaneous full-weekend itineraries based on mood, interests, and budget.",
      instruction="""
        You are the "Spontaneous Weekend Trip" Generator 🧳 – a specialized AI assistant that creates engaging full-weekend itineraries.

            Your Mission:
            Transform a simple mood or interest into a complete 2-day or 3-day weekend trip, with real-time details, while respecting a budget.

            Guidelines:
            1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
            2. **Weekend Structure**: Create plans for each day (e.g., Saturday and Sunday), dividing them into morning, afternoon, and evening.
            3. **Real-Time Focus**: Search for current operating hours and special events.
            4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN the itinerary in MARKDOWN FORMAT with day labels (e.g., **Friday**, **Saturday**, **Sunday**) and time blocks with specific venue names.
       """,
      tools=[google_search]
  )
weekend_trip_agent = create_weekend_trip_agent()
print(f"🧞 Agent '{weekend_trip_agent.name}' is created and ready for adventure!")

🧞 Agent 'weekend_trip_agent' is created and ready for adventure!


In [8]:
# --- A Helper Function to Run Our Agents ---
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.

session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"


🧠 Explanation: What does this function do?

This function is responsible for running a user query through a specific agent in a session and then showing the agent's final response nicely formatted in Markdown. It helps us see not only the final answer, but also how the agent "thinks" step by step.

🔑 Key Concepts:

🏃‍♀️ Runner
The Runner is the engine that manages the conversation between the user and the agent. It handles the entire flow: sending the question, tracking the steps the agent takes (called events), and collecting the final answer.

🔄 Async Event Loop
The runner.run_async(...) function returns a stream of asynchronous events.
Each event represents something the agent is doing — like thinking, calling a tool, or responding.
The line async for event in runner.run_async(...) loops through each of these events in real time, as they happen.

This lets us follow the agent’s reasoning step by step.
Think of it like watching the agent "work things out" before answering.

👀 is_router flag
This optional flag controls whether we want to print the internal steps (events) of the agent.

When is_router = False (default), we print each event — useful when learning or debugging.

When is_router = True, we hide those steps. This is useful when an agent is being used as a tool by another agent (like a sub-agent), and we don’t want to clutter the logs.

✅ Final Output
The agent eventually sends a special event called a final response.
When that happens, we extract just the final message and display it using Markdown (to look nice in notebooks).

🧪 Why is this useful?
This pattern is great for testing and learning how agents behave.
It gives us insight into the "thinking process" of the model, not just the end result.

In [ ]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=weekend_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a relaxing and interesting weekend trip in Venecia, Italy. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(weekend_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

🗣️ User Query: 'Plan a relaxing and interesting weekend trip in Venecia, Italy. Keep it affordable!'

🚀 Running query for agent: 'weekend_trip_agent' in session: '310b65c6-6e93-4a01-bc3f-6e0afd3f8055'...
